
# Data acquisition controllers
<br>
<br>
<img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" />
Sardana-Training by ALBA Synchrotron is licensed under the Creative Commons Attribution 4.0 International License.  
To view a copy of this license, visit http://creativecommons.org/licenses/by/4.0/.

## Contents

* Writing counter/timer controller - Network Trafic Counter
* Playing with Blender Detector - 2D experimental channel controller

## Network Traffic Counter
### How to write a counter/timer controller

### Network Traffic Counter

* Open the code in your favourite editor e.g.:
  ```
  kwrite ~/sardana-training/controllers/templates/NetworkTrafficCTCtrl.py
  ```

### Network Traffic Counter
* Network traffic counter counts bytes that pass via a network interface (both ways)
* Demo:
 * Directly from the system:
   ```
   cat /proc/net/dev | grep eth0
   ```
 * Using python function:
   ```
   cd ~/sardana-training/controllers
   python3 -c "from NetworkTrafficCTCtrl import read_network_counts; print(read_network_counts('eth0'))"
   ```

### [Writing constructor](http://www.sardana-controls.org/devel/howto_controllers/howto_controller.html#constructor)
* Constructor
 * Called on: controllers creation, pool startup and controller's code reload
 * Accepts arguments: instance (name of the controller instance) and properties (dictionary with the controller properties)
 * If an exception is raised when constructing the controller, the controller automatically gets into the Fault state and its status describes the exception that occured
* [Define controller property](http://www.sardana-controls.org/devel/howto_controllers/howto_controller.html#extra-controller-properties) `interface`

### [Writing constructor](http://www.sardana-controls.org/devel/howto_controllers/howto_controller.html#constructor)
* Counter will work on the following principle:
 * Load of the timer will store the integration time in the cache
 * Start of the counting will latch the current number of bytes
 * Start of the counting will define the end acquisition time (current time + integration time)
* In the constructor we need to instantiate the necessary variables:
 * acq_time = 1
 * acq_end_time = time.time()
 * start_counts = 0

### Instantiate controller
* Deploy controller:
  ```
  ln -s ~/sardana-training/controllers/templates/NetworkTrafficCTCtrl.py ~/controllers
  ```
* Load it in the system (in Spock):
  ```
  addctrllib NetworkTrafficCTCtrl
  ```
* Create an instance of the controller:
  ```
  defctrl NetworkTrafficCounterTimerController netctrl interface eth0
  ```
* Ask for controllers state:
  ```
  netctrl.state()
  ```

### [Implement StateOne](http://www.sardana-controls.org/devel/howto_controllers/howto_controller.html#get-axis-state)
* Assume state is On
* Only if current time is less then the acq_end_time return Moving state
* IMPORTANT: due to [sardana-org/sardana#621](https://github.com/sardana-org/sardana/issues/621) return an empty status `""`

### [Implement LoadOne](http://www.sardana-controls.org/devel/howto_controllers/howto_countertimercontroller.html#load-a-counter)
 * receives either integration time or monitor counts (negative number) as arguments \*
 * is called only on the controller's timer or monitor
 
 * store the integration time in the cache `acq_time`
 
\* An advanced API is available for continuous acquisitions e.g. continuous scans

### [Implement ReadOne](http://www.sardana-controls.org/devel/howto_controllers/howto_countertimercontroller.html#get-counter-value)
 * returns a single counter value
 * is called multiple times during the acquisition operation

 * latch network bytes
 * return a difference between the current network bytes and `start_counts`
 
\* An advanced API is available for continuous acquisitions e.g. continuous scans

### Instantiate net counter
* Reload controller code (in Spock):
  ```
  relctrlcls NetworkTrafficCounterTimerController
  ``` 
* Create motor instance (in Spock):
  ```
  defelem net netctrl 1
  ```
* Ask for motor state (in Spock):
  ```
  net.state()
  ```

### [Implement StartOne](http://www.sardana-controls.org/devel/howto_controllers/howto_countertimercontroller.html#start-a-counter)
 * receives intergation timer or monitor count as argument
 * latch the current bytes to `start_counts` and calcultate the end acquisition time in `acq_end_time`

### Demonstrate single acquisition with net counter
* Reload controller code (in Spock):
  ```
  relctrlcls NetworkTrafficCounterTimerController
  ``` 
* Count (in Spock)  -> zero counts!:
  ```
  uct 1 net
  ```
* While long counting (in Spock):
  ```
  uct net 10
  ```
  get sardana CHANGELOG with the `wget` tool:
  ```
  wget https://github.com/sardana-org/sardana/blob/develop/CHANGELOG.md
  ```


## Blender Detector
### Play with the 2D experimental channel on example of Blender Detector

### Instantiate controller and experimental channels
* Deploy controller:
  ```
  ln -s ~/sardana-training/controllers/Blender2DCtrl.py ~/controllers
  ```
* Load it in the system (in Spock):
  ```
  addctrllib Blender2DCtrl
  ```
* Create an instance of the controller (in Spock):
  ```
  defctrl Blender2DController ble2dctrl
  ```
* Ask for controllers state (in Spock): 
  ```
  ble2dctrl.state()
  ```
* Create an instance of the 2D experimental channel (in Spock): 
  ```
  defelem ble2d ble2dctrl
  ```
* Ask for channels state (in Spock):
  ```
  ble2d.state()
  ```

### Acquire with the detector - data transfer

* Configure timer (in Spock, line by line):
  ```
  ble2d.timer = "__self"
  ct 1 ble2d
  ```
* Show image (`value` attribute): 
  ```
  taurus form ble2d
  ```
* In `expconf` (started from Spock):
  * define measurement group with the `ble2d` channel
  * configure storage to HDF5 file (_Storage_ tab -> _File name(s)_ & _Path_):
* Move slits to a well known position (in Spock):
  ```
  mv gapvert 10 gaphor 10 offsetvert 0 offsethor 0
  ```
* Slightly open left blade (in Spock):
  ```
  mv left 50
  ```
* Scan the vertical offset (in Spock):
  ```
  dscan offsetvert -10 10 10 0.1
  ```
* Show images with `silx` (remember to close `silx` afterwards):
  ```
  silx view
  ```

### Acquire with the detector - data references transfer

* In `expconf` configure data referencing:
    * _Ref Enabled_ -> True
    * _Ref Pattern_ -> `h5file:///tmp/ble2d_img{image_nb:03d}.h5`
* Repeat the last scan (in Spock):
  ```
  dscan offsetvert -10 10 10 0.1
  ```
* Show images via VDS with `silx` (remember to close `silx` afterwards):
  ```
  silx view
  ```
  * Note the usage of HDF5 VDS - image data are not stored in the Sardana file. 
    